In [1]:
from PySide6.QtCore import QByteArray

In [ ]:
QByteArray(b"hello")

In [2]:
loader = LoadShotParameters(which="all")


with experiment.storage_session() as session:
    sequence = session.get_sequence(r"\new folder\new sequence")
    shot = list(sequence.get_shots())[0]
    %timeit loader(shot)

1.29 ms ± 56.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
from collections.abc import Mapping
from typing import Literal, Any

import attrs
import polars

from caqtus.session import Shot
from caqtus.types.variable_name import DottedVariableName
from ._combinable_importers import CombinableLoader


@attrs.define
class LoadShotParameters(CombinableLoader):
    """Loads the parameters of a shot.

    When it is evaluated on a shot, it returns a polars dataframe with a single row and
    with several columns named after each parameter requested.

    If some parameters are quantity with units, the dtype of the associated column will
    be a quantity dtype with two fields, magnitude and units.

    Parameters:
        which: The parameters to load from a shot.

            If it is "sequence", only the parameters defined at the sequence level are
            loaded.
            If "all", both sequence specific and global parameters are loaded.
    """

    which: Literal["sequence", "all"] = "sequence"

    @staticmethod
    def _parameters_to_dataframe(
        parameters: Mapping[DottedVariableName, Any]
    ) -> polars.DataFrame:
        series: list[polars.Series] = []

        for parameter_name, value in parameters.items():
            name = str(parameter_name)
            match value:
                case [float(magnitude), str(unit)]:
                    s = polars.Series(
                        name,
                        [
                            polars.Series(
                                "magnitude", [magnitude], dtype=polars.Float64
                            ),
                            polars.Series("units", [unit], dtype=polars.Categorical),
                        ],
                        dtype=polars.Struct,
                    )
                case int():
                    s = polars.Series(name, [value], dtype=polars.Int64)
                case float():
                    s = polars.Series(name, [value], dtype=polars.Float64)
                case bool():
                    s = polars.Series(name, [value], dtype=polars.Boolean)
                case _:
                    raise TypeError(
                        f"Unsupported type {type(value)} for parameter {name}."
                    )
            series.append(s)
        series.sort(key=lambda s: s.name)
        dataframe = polars.DataFrame(series)
        return dataframe

    def load(self, shot: Shot) -> polars.DataFrame:
        """Load the parameters of a shot."""

        parameter_names = shot.sequence.get_parameter_names(self.which)

        unstructured_parameters = (
            shot._session.sequences.load_unstructured_shot_parameters(
                shot.sequence.path, shot.index, parameter_names
            )
        )

        return self._parameters_to_dataframe(unstructured_parameters)


{DottedVariableName('rep'): 0,
 DottedVariableName('time'): <Quantity(0.0, 'microsecond')>}